##Function
The objective of using this Jupyter notebook is experimentation and testing of the different models that will be developed in order to pick out the best one.


In [2]:
#Implementation 1
#Checking for performance after preprocessing 
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate



train_df = pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/train_clean.csv")
test_df =  pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/test_clean.csv")

#X_train, y_train and X_test
y_train = train_df["bank_account"]
X_train = train_df.drop(["bank_account"], axis=1)
X_test = test_df.drop(["bank_account"], axis=1)

#Dict of classifiers
classifiers = {
    "KNN" : KNeighborsClassifier(),
    "LR" : LogisticRegression(max_iter=1000),
    "MLP" : MLPClassifier(max_iter=1000),
    "XGB" : XGBClassifier(),
    "LGB" : LGBMClassifier(),
    "SVC" : SVC(),
    "DT" : DecisionTreeClassifier(),
    "RT" : RandomForestClassifier()
}

results = pd.DataFrame(columns=["Classifier", "Avg_Accuracy", "Avg_F1_Score"])

for name, clf in classifiers.items():
    model = clf
    cv_results = cross_validate(model, X_train, y_train, cv=10, 
                 scoring=(['accuracy', 'f1']))

    results = results.append({
        "Classifier": name, 
        "Avg_Accuracy": cv_results["test_accuracy"].mean(),
        "Avg_F1_Score" : cv_results["test_f1"].mean()
    }, ignore_index=True)

results["Avg_Overall"] = (results["Avg_Accuracy"] + results["Avg_F1_Score"])/2
results = results.sort_values("Avg_Overall", ascending=False)
results

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/mwanikii/.local/lib/python3.10/site-packages/sklearn/neigh

,Classifier,Avg_Accuracy,Avg_F1_Score,Avg_Overall
3,XGB,0.873536,0.451148,0.662342
4,LGB,0.877149,0.441471,0.659310
2,MLP,0.871366,0.432076,0.651721
1,LR,0.874172,0.410026,0.642099
7,RT,0.854451,0.423872,0.639162
6,DT,0.818998,0.393227,0.606112
0,KNN,0.858403,0.347177,0.602790
5,SVC,0.859208,0.000000,0.429604


In the above, XGB and LGB have displayed the best Avg_Overall score. In the next attempt, the models will be designed to take in categorical data as it is rather than encoding it.
NB: This is when the date column has not been removed

When the date column has been dropped, LGB is replaced by MLP while XGB maintains its position. 


The next cell will use the parameters in the file training_xgboost.ipynb to come up with a prediction.

In [8]:
#Checking the result of XGBoost with the results provided by Optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
#Train_test_splits to test model improvement by comparing y_val and y_preds
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=100, test_size=0.1)

clf =  XGBClassifier(n_estimators=10000,
		learning_rate=0.20172781199546924,
		max_leaves=2560,
		max_depth=8,
		max_bin=280,
		reg_lambda=40,
		alpha=5,
		gamma=3.3910506390918242)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

print("Error rate: ", 1 - accuracy_score(y_val, y_pred))


Error rate:  0.10623229461756378


In [9]:
clf = XGBClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

print("Error rate: ", 1 - accuracy_score(y_val, y_pred))


Error rate:  0.10953729933899903


##Using Catboost
I will now attempt to incorporate the most optimal paramters as indicated by optuna and check for performance in comparison to xgboost


In [5]:
#Using catboost
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

train_df = pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/Train.csv")
test_df = pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/Test.csv")

#Encoding the bank_account column
LE = LabelEncoder()
train_df["bank_account"] = LE.fit_transform(train_df["bank_account"])

#X_train, X_test, y_train, y_test
X_train = train_df.drop(["bank_account", "uniqueid", "year"], axis=1)
y_train = train_df["bank_account"]

X_test = test_df.drop(["year", "uniqueid"], axis=1)

#Train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=100)


cat_features = np.where(X_train.dtypes != np.int64)[0]

#Training using Optuna parameters
clf = CatBoostClassifier(iterations=10000,
		learning_rate=0.2979761615880641,
		l2_leaf_reg=100,
		depth=3,
        cat_features=cat_features)

clf.fit(X_train, y_train)
y_preds = clf.predict(X_val)
print("Error rate: ", 1 - accuracy_score(y_val, y_preds))


0:	learn: 0.4957593	total: 9.74ms	remaining: 1m 37s
1:	learn: 0.3975757	total: 20.1ms	remaining: 1m 40s
2:	learn: 0.3520658	total: 30.6ms	remaining: 1m 42s
3:	learn: 0.3276283	total: 40.1ms	remaining: 1m 40s
4:	learn: 0.3138623	total: 51.4ms	remaining: 1m 42s
5:	learn: 0.3044774	total: 61ms	remaining: 1m 41s
6:	learn: 0.2988553	total: 69.4ms	remaining: 1m 39s
7:	learn: 0.2960509	total: 76.8ms	remaining: 1m 35s
8:	learn: 0.2933411	total: 87.1ms	remaining: 1m 36s
9:	learn: 0.2907239	total: 95.6ms	remaining: 1m 35s
10:	learn: 0.2896244	total: 104ms	remaining: 1m 34s
11:	learn: 0.2878613	total: 112ms	remaining: 1m 33s
12:	learn: 0.2864073	total: 121ms	remaining: 1m 32s
13:	learn: 0.2856210	total: 132ms	remaining: 1m 34s
14:	learn: 0.2851363	total: 140ms	remaining: 1m 32s
15:	learn: 0.2847631	total: 150ms	remaining: 1m 33s
16:	learn: 0.2843190	total: 160ms	remaining: 1m 33s
17:	learn: 0.2838420	total: 170ms	remaining: 1m 34s
18:	learn: 0.2837171	total: 179ms	remaining: 1m 34s
19:	learn: 0.2

xgboost gives better perormance than catboost as displayed above.